# Uptrained Neural Network

## Notebook Description

In this Notebook we will uptrain our Neural Network on an increased number of Jobs.<br>
If this is the first iteration, we will uptrain the Supervised Neural Network to 9 Jobs.<br>
If he have uptrained it already before on more than 8 Jobs, we increase the maximum number of Jobs by one and continue training that uptrained version.

## Code

In [1]:
import numpy as np
import os
import pickle
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, LSTM, Concatenate, LeakyReLU, Softmax, Dropout
from keras.layers import Lambda, Flatten, Bidirectional, TimeDistributed, Reshape, MultiHeadAttention, LayerNormalization
from keras.activations import tanh
from keras.models import Model, Sequential
import keras.backend
import random
from copy import copy
#import necessary notebooks
import import_ipynb
from Jobs_and_Machines import *
from States_and_Policies import *
from Action_Pointer import *

importing Jupyter notebook from Jobs_and_Machines.ipynb
importing Jupyter notebook from States_and_Policies.ipynb
importing Jupyter notebook from Global_Variables.ipynb
importing Jupyter notebook from Action_Pointer.ipynb


In [2]:
#change working directory
os.chdir('D:\\Job-Scheduling-Files')
os.getcwd()

'D:\\Job-Scheduling-Files'

### Create Estimated Training Data

Set the increased number of Jobs.

In [3]:
n_estim = int(input("What is the new number of Jobs?\n"))

n = 8
n_min = 3

m = 4
m_min = 2

DS_max = 10
DS_min = 1

What is the new number of Jobs?
12


In [4]:
def load_data(DS_min,DS_max):
    
    #dictionary will contain data from all respective data sets
    data_dict = dict(((n_state,m_state),[[],[]]) 
                           for n_state in range(n_min,n+1) for m_state in range(m_min,m+1))
    
    #load data
    for n_state in range(n_min, n+1):
        for m_state in range(m_min, m+1):
            #resource information of the jobs: processing time on every machine and its metadata
            x_res = []
            #urgency information of the jobs: earliness and weight
            x_urg = []
            #targets
            y = []
            #loop over every data set "DS"
            for DS in ["0"*(2-len(str(i)))+str(i) for i in range(DS_min,DS_max+1)]:
                training_data_path = f'Data/DataSet_{DS}/LSTM_Data_RR_{DS}/{n_state}-jobs-{m_state}-machines_{DS}.pickle'
                with open(training_data_path, 'rb') as f:
                    #load data set of key (n_state,m_state)
                    data = pickle.load(f)
                    #get data about its resources for every job of one state of one Job Scheduling Problem
                    jobs_resources = data[0][0][:,:,:-2].reshape((-1,n_state,m_state,4))
                    #get data abouts its urgenicies for every job  of one state of one Job Scheduling Problem
                    jobs_urgencies = data[0][0][:,:,-2:]
                    #add to data of states of other Job Scheduling Problems
                    x_res.append(jobs_resources)
                    x_urg.append(jobs_urgencies)
                    y.append(data[1][0])
            #transform list of data from states of Job Scheduling Problems to numpy array and add to final data dictionary
            data_dict[(n_state,m_state)][0].append(np.concatenate(x_res))
            data_dict[(n_state,m_state)][0].append(np.concatenate(x_urg))
            data_dict[(n_state,m_state)][1].append(np.concatenate(y))
            
    return data_dict

We will now load the estimated data and transform it into the form of training data.<br>
Note that the path in the function might have to be changed by user.

In [5]:
#load estimated data
def load_estim_data():
    
    #dictionary will contain all estimated
    estim_data_dict = dict(((n_state,m_state),[[],[]]) 
                           for n_state in range(n+1, n_estim+1) for m_state in range(m_min,m+1))
    
    #load estimated data from n+1 to n_estim
    for n_state in range(n, n_estim+1):
        for m_state in range(m_min, m+1):
            #resource information of the jobs: processing time on every machine and its metadata
            x_res = []
            #urgency information of the jobs: earliness and weight
            x_urg = []
            #targets
            y = []
            """use your own path here"""
            estim_data_path = f'Data/EstimData/{n_state}_Jobs/LSTM_EstimData_RR/{n_state}-jobs-{m_state}-machines.pickle'
            with open(estim_data_path, 'rb') as f:
                data = pickle.load(f)
                #get data about its resources for every job of one state of one Job Scheduling Problem
                jobs_resources = data[0][0][:,:,:-2].reshape((-1,n_state,m_state,4))
                #get data abouts its urgenicies for every job  of one state of one Job Scheduling Problem
                jobs_urgencies = data[0][0][:,:,-2:]
                #add to data of states of other Job Scheduling Problems
                x_res.append(jobs_resources)
                x_urg.append(jobs_urgencies)
                y.append(data[1][0])

            #transform list of data from states of Job Scheduling Problems to numpy array and add to final data dictionary
            estim_data_dict[(n_state,m_state)][0].append(np.concatenate(x_res))
            estim_data_dict[(n_state,m_state)][0].append(np.concatenate(x_urg))
            estim_data_dict[(n_state,m_state)][1].append(np.concatenate(y))

    return estim_data_dict

Load the estimated data and bring it into the correct form by splitting it every <i>(n_state,m_state)</i>-subset from it into a list of inputs and a list of targets.

In [6]:
estim_dict = load_estim_data()

In [7]:
x_estim_list, y_estim_list = [], []
for key in estim_dict:
    
    #estimated data
    x_estim, y_estim = estim_dict[key]
    x_estim_list.append(x_estim)
    y_estim_list.append(y_estim)

Load Network.

In [ ]:
#load desired version of Neural Network
def load_NN(NN_name):
    """path might have to be updated based on users storage"""
    path = 'D:\\Job-Scheduling-Files\'
    
    NN = keras.models.load_model(f'{path}{NN_name}.h5', custom_objects={'FeedForward': FeedForward, 'Pointer': Pointer, 'MSE_with_Softmax': MSE_with_Softmax, 'costs':costs})
    NN.run_eagerly = True
    
    return NN

In [9]:
#Uptrained_Neural_Network = load_NN(f"Final_Pointer3")
Uptrained_Neural_Network = load_NN(f"Uptrained_Final_Pointer_{n_estim-1}_Jobs")

### Compile Model

Compile the model the same way as before.

In [12]:
learning_rate = 0.001

Neural_Network.compile(
        #custom loss
        loss = MSE_with_Softmax,
        #optimizer
        optimizer = keras.optimizers.Adam(learning_rate = learning_rate),
        run_eagerly=True,
        #custom metric
        metrics = [costs])

### Uptraining

We will now use the estimated data to uptrain the Neural Network.<br>
Again, one epoch will consist of the states related to one <i>(n_state,m_state)</i>-subset only.<br>
Since is has been trained for any number of Jobs greater than <i>n</i> but lower than <i>n_estim</i> already, we will first train it on the data of <i>n_estim</i> jobs. We loop 5 times through the data, resulting in 5*3 epochs.

In [13]:
print(f'We train for {n_estim} Jobs')
ep=0*3
#load the last three subsets, as these are (n_estim,4), (n_estim,3) and (n_estim,2)
for j in range(len(x_estim_list)-3,len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

We train for 12 Jobs
782/782 [==============================] - 1847s 2s/step - loss: 0.2005 - costs: 0.2665
Epoch 2/2
782/782 [==============================] - 554s 708ms/step - loss: 0.4380 - costs: 0.6844
Epoch 3/3
782/782 [==============================] - 564s 721ms/step - loss: 0.8190 - costs: 1.2095


In [14]:
print(f'We train for {n_estim} Jobs')
ep=1*3
#load the last three subsets, as these are (n_estim,4), (n_estim,3) and (n_estim,2)
for j in range(len(x_estim_list)-3,len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

We train for 12 Jobs
Epoch 4/4
782/782 [==============================] - 532s 680ms/step - loss: 0.1972 - costs: 0.2604
Epoch 5/5
782/782 [==============================] - 547s 700ms/step - loss: 0.4212 - costs: 0.6698
Epoch 6/6
782/782 [==============================] - 564s 721ms/step - loss: 0.8041 - costs: 1.1892


In [15]:
print(f'We train for {n_estim} Jobs')
ep=2*3
#load the last three subsets, as these are (n_estim,4), (n_estim,3) and (n_estim,2)
for j in range(len(x_estim_list)-3,len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

We train for 12 Jobs
Epoch 7/7
782/782 [==============================] - 534s 683ms/step - loss: 0.1932 - costs: 0.2596
Epoch 8/8
782/782 [==============================] - 549s 702ms/step - loss: 0.4184 - costs: 0.6650
Epoch 9/9
782/782 [==============================] - 564s 722ms/step - loss: 0.7956 - costs: 1.1858


In [16]:
print(f'We train for {n_estim} Jobs')
ep=3*3
#load the last three subsets, as these are (n_estim,4), (n_estim,3) and (n_estim,2)
for j in range(len(x_estim_list)-3,len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

We train for 12 Jobs
Epoch 10/10
782/782 [==============================] - 843s 1s/step - loss: 0.1930 - costs: 0.2555
Epoch 11/11
782/782 [==============================] - 554s 709ms/step - loss: 0.4143 - costs: 0.6491
Epoch 12/12
782/782 [==============================] - 566s 724ms/step - loss: 0.7870 - costs: 1.1666


In [17]:
print(f'We train for {n_estim} Jobs')
ep=4*3
#load the last three subsets, as these are (n_estim,4), (n_estim,3) and (n_estim,2)
for j in range(len(x_estim_list)-3,len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

We train for 12 Jobs
Epoch 13/13
782/782 [==============================] - 536s 685ms/step - loss: 0.1914 - costs: 0.2551
Epoch 14/14
782/782 [==============================] - 549s 702ms/step - loss: 0.4114 - costs: 0.6469
Epoch 15/15
782/782 [==============================] - 577s 738ms/step - loss: 0.7891 - costs: 1.1619


If <i>n_estim > 9</i>, we retrain the Network on the entire estimated data set. One loop therefore consists of 3*i epochs.<br>
We do <i>n_estim-n+1</i> loops.

In [19]:
print(f'We train for {n_estim} Jobs')
ep=5*3
for j in range(len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

We train for 12 Jobs
Epoch 16/16
782/782 [==============================] - 475s 607ms/step - loss: 0.3298 - costs: 0.2496
Epoch 17/17
782/782 [==============================] - 493s 631ms/step - loss: 0.8400 - costs: 0.5915
Epoch 18/18
782/782 [==============================] - 506s 647ms/step - loss: 1.4663 - costs: 0.9090
Epoch 19/19
782/782 [==============================] - 10812s 14s/step - loss: 0.2588 - costs: 0.2532
Epoch 20/20
782/782 [==============================] - 1630s 2s/step - loss: 0.6463 - costs: 0.6338
Epoch 21/21
782/782 [==============================] - 529s 676ms/step - loss: 1.1543 - costs: 1.0206
Epoch 22/22
782/782 [==============================] - 516s 659ms/step - loss: 0.2344 - costs: 0.2727
Epoch 23/23
782/782 [==============================] - 532s 680ms/step - loss: 0.5311 - costs: 0.6871
Epoch 24/24
782/782 [==============================] - 549s 702ms/step - loss: 0.9620 - costs: 1.1236
Epoch 25/25
782/782 [==============================] - 544s 695

In [20]:
print(f'We train for {n_estim} Jobs')
ep=5*3 + len(x_estim_list)
for j in range(len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

We train for 12 Jobs
Epoch 28/28
782/782 [==============================] - 481s 616ms/step - loss: 0.3215 - costs: 0.2416
Epoch 29/29
782/782 [==============================] - 487s 623ms/step - loss: 0.8265 - costs: 0.5744
Epoch 30/30
782/782 [==============================] - 516s 660ms/step - loss: 1.4457 - costs: 0.9109
Epoch 31/31
782/782 [==============================] - 500s 639ms/step - loss: 0.2534 - costs: 0.2544
Epoch 32/32
782/782 [==============================] - 1133s 1s/step - loss: 0.6399 - costs: 0.6263
Epoch 33/33
782/782 [==============================] - 510s 652ms/step - loss: 1.1443 - costs: 1.0105
Epoch 34/34
782/782 [==============================] - 515s 659ms/step - loss: 0.2343 - costs: 0.2693
Epoch 35/35
782/782 [==============================] - 528s 675ms/step - loss: 0.5283 - costs: 0.6822
Epoch 36/36
782/782 [==============================] - 554s 708ms/step - loss: 0.9493 - costs: 1.1060
Epoch 37/37
782/782 [==============================] - 539s 689

In [22]:
print(f'We train for {n_estim} Jobs')
ep=5*3 + 2*len(x_estim_list)
for j in range(len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

We train for 12 Jobs
Epoch 40/40
782/782 [==============================] - 277s 354ms/step - loss: 0.3210 - costs: 0.2446
Epoch 41/41
782/782 [==============================] - 320s 409ms/step - loss: 0.8180 - costs: 0.5744
Epoch 42/42
782/782 [==============================] - 328s 420ms/step - loss: 1.4365 - costs: 0.8985
Epoch 43/43
782/782 [==============================] - 343s 438ms/step - loss: 0.2556 - costs: 0.2492
Epoch 44/44
782/782 [==============================] - 364s 465ms/step - loss: 0.6369 - costs: 0.6227
Epoch 45/45
782/782 [==============================] - 332s 424ms/step - loss: 1.1354 - costs: 1.0137
Epoch 46/46
782/782 [==============================] - 299s 382ms/step - loss: 0.2330 - costs: 0.2727
Epoch 47/47
782/782 [==============================] - 308s 394ms/step - loss: 0.5285 - costs: 0.6851
Epoch 48/48
782/782 [==============================] - 318s 407ms/step - loss: 0.9444 - costs: 1.1141
Epoch 49/49
782/782 [==============================] - 312s 3

In [25]:
print(f'We train for {n_estim} Jobs')
ep=5*3 + 3*len(x_estim_list)
for j in range(len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

We train for 12 Jobs
Epoch 52/52
782/782 [==============================] - 273s 348ms/step - loss: 0.3169 - costs: 0.2413
Epoch 53/53
782/782 [==============================] - 285s 364ms/step - loss: 0.8231 - costs: 0.5726
Epoch 54/54
782/782 [==============================] - 292s 374ms/step - loss: 1.4126 - costs: 0.8937
Epoch 55/55
782/782 [==============================] - 287s 367ms/step - loss: 0.2553 - costs: 0.2479
Epoch 56/56
782/782 [==============================] - 296s 379ms/step - loss: 0.6341 - costs: 0.6293
Epoch 57/57
782/782 [==============================] - 306s 391ms/step - loss: 1.1296 - costs: 1.0089
Epoch 58/58
782/782 [==============================] - 299s 383ms/step - loss: 0.2309 - costs: 0.2719
Epoch 59/59
782/782 [==============================] - 309s 396ms/step - loss: 0.5265 - costs: 0.6864
Epoch 60/60
782/782 [==============================] - 319s 407ms/step - loss: 0.9427 - costs: 1.0996
Epoch 61/61
782/782 [==============================] - 315s 4

In [28]:
print(f'We train for {n_estim} Jobs')
ep=5*3 + 4*len(x_estim_list)
for j in range(len(x_estim_list)):
    #train one epoch on  every subset
    history = Uptrained_Neural_Network.fit(x_estim_list[j], y_estim_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
    ep = history.epoch[-1]+1

We train for 12 Jobs
Epoch 64/64
782/782 [==============================] - 458s 585ms/step - loss: 0.3117 - costs: 0.2405
Epoch 65/65
782/782 [==============================] - 507s 648ms/step - loss: 0.8429 - costs: 0.5788
Epoch 66/66
782/782 [==============================] - 6947s 9s/step - loss: 1.4117 - costs: 0.8779
Epoch 67/67
782/782 [==============================] - 3342s 4s/step - loss: 0.2493 - costs: 0.2465
Epoch 68/68
782/782 [==============================] - 523s 668ms/step - loss: 0.6341 - costs: 0.6203
Epoch 69/69
782/782 [==============================] - 549s 702ms/step - loss: 1.1254 - costs: 0.9920
Epoch 70/70
782/782 [==============================] - 547s 699ms/step - loss: 0.2310 - costs: 0.2662
Epoch 71/71
782/782 [==============================] - 566s 724ms/step - loss: 0.5242 - costs: 0.6855
Epoch 72/72
782/782 [==============================] - 580s 741ms/step - loss: 0.9385 - costs: 1.0960
Epoch 73/73
782/782 [==============================] - 579s 741ms

Finally, we save the uptrained version of the Neural Network.

In [30]:
Uptrained_Neural_Network.save(f'Uptrained_Neural_Network_{n_estim}_Jobs.h5')